In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Mar  1 23:11:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(12)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    #scheduler = ReduceLROnPlateau(optimizer, factor=0.9)
    #scheduler.step(test_loss)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(1,16,3,padding=0,bias=False),        #conv1  1 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:3
                                 nn.BatchNorm2d(16),                            #o/p:26
                                 nn.Dropout2d(0.04))


        self.conv2=nn.Sequential(nn.Conv2d(16,24,3,padding=0,bias=False),       #conv2  16 input , 32 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:5
                                 nn.BatchNorm2d(24),                            #o/p:24
                                 nn.Dropout2d(0.04))




        # used 1x1 kernel
        self.conv3=nn.Sequential(nn.Conv2d(24,12,1,bias=False))                  #conv3  32 input , 8 0utput and 1x1 kernel
                                                                                #RF:5
                                                                                #o/p:24



        self.maxpool=nn.MaxPool2d(2,2)                                          # maxpooling 2x2 kernel and stride 2
                                                                                #RF:6
                                                                                #o/p:12




        self.conv4=nn.Sequential(nn.Conv2d(12,16,3,padding=0,bias=False),        #conv4  8 input , 14 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:10
                                 nn.BatchNorm2d(16),                            #o/p:10
                                 nn.Dropout2d(0.04))


        self.conv5=nn.Sequential(nn.Conv2d(16,24,3,padding=0,bias=False),       #conv5  14 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:14
                                 nn.BatchNorm2d(24),                            #o/p:8
                                 nn.Dropout2d(0.04))



        #self.maxpool=nn.MaxPool2d(2,2)

        self.conv6=nn.Sequential(nn.Conv2d(24,20,1,padding=0,bias=False),
                                 nn.ReLU(),
                                 nn.BatchNorm2d(20),
                                 nn.Dropout2d(0.04)      )       #conv6  16 input , 10 0utput and 1x1 kernel
                                                                                #RF:14
                                                                                #o/p:8


        #self.conv7=nn.Sequential(nn.Conv2d(16,10,3,padding=0,bias=False),
                                 #nn.ReLU(),                                     #conv6  16 input , 10 0utput and 3x3 kernel
                                 #nn.BatchNorm2d(16),                            #RF:18
                                 #nn.Dropout2d(0.04))                            #o/p:6


        self.GAP=nn.AvgPool2d(8)                                                # Gobal Average Pooling
                                                                                #RF:28
                                                                                #o/p:1
        #self.flatten_gap = nn.Sequential(nn.AvgPool2d(8), nn.Flatten())
        #self.final_linear = nn.Sequential(nn.Linear(20,10))


        self.conv8 = nn.Sequential(nn.Conv2d(20,10,1, padding=0, bias=False))   #RF:28
                                                                                #O/P:1




    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        #x=self.maxpool(x)
        x=self.conv3(x)
        x=self.maxpool(x)
        x=self.conv4(x)
        x=self.conv5(x)
        #x=self.maxpool(x)
        x=self.conv6(x)
        #x=self.conv7(x)
        x = self.GAP(x)
        #x=self.flatten_gap(x)
        #x=self.final_linear(x)
        x=self.conv8(x)
        x=x.view(-1,10)
        return F.log_softmax(x)

#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 24, 24, 24]           3,456
              ReLU-6           [-1, 24, 24, 24]               0
       BatchNorm2d-7           [-1, 24, 24, 24]              48
         Dropout2d-8           [-1, 24, 24, 24]               0
            Conv2d-9           [-1, 12, 24, 24]             288
        MaxPool2d-10           [-1, 12, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,728
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
        Dropout2d-14           [-1, 16,

<ipython-input-8-9f632a931c65>:90: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("epoches:",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoches: 1


  0%|          | 0/1875 [00:00<?, ?it/s]<ipython-input-8-9f632a931c65>:90: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Loss=0.07545950263738632 Batch_id=1874 Accuracy=90.31: 100%|██████████| 1875/1875 [00:34<00:00, 54.04it/s]



Test set: Average loss: 0.0561, Accuracy: 9838/10000 (98.38%)

epoches: 2


Loss=0.024293385446071625 Batch_id=1874 Accuracy=96.90: 100%|██████████| 1875/1875 [00:34<00:00, 53.93it/s]



Test set: Average loss: 0.0393, Accuracy: 9882/10000 (98.82%)

epoches: 3


...


Test set: Average loss: 0.0231, Accuracy: 9932/10000 (99.32%)

epoches: 18


Loss=0.06360456347465515 Batch_id=1874 Accuracy=98.59: 100%|██████████| 1875/1875 [00:33<00:00, 56.34it/s]



Test set: Average loss: 0.0214, Accuracy: 9930/10000 (99.30%)

epoches: 19


Loss=0.0034810518845915794 Batch_id=1874 Accuracy=98.60: 100%|██████████| 1875/1875 [00:33<00:00, 55.72it/s]



Test set: Average loss: 0.0209, Accuracy: 9928/10000 (99.28%)



In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(1,16,3,padding=0,bias=False),        #conv1  1 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:3
                                nn.BatchNorm2d(16),                            #o/p:26
                                 nn.Dropout2d(0.04))


        self.conv2=nn.Sequential(nn.Conv2d(16,20,3,padding=0,bias=False),       #conv2  16 input , 32 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:5
                                 nn.BatchNorm2d(20),                            #o/p:24
                                 nn.Dropout2d(0.04))




        # used 1x1 kernel
        self.conv3=nn.Sequential(nn.Conv2d(20,12,1,bias=False))                  #conv3  32 input , 8 0utput and 1x1 kernel
                                                                                #RF:5
                                                                                #o/p:24



        self.maxpool=nn.MaxPool2d(2,2)                                          # maxpooling 2x2 kernel and stride 2
                                                                                #RF:6
                                                                                #o/p:12




        self.conv4=nn.Sequential(nn.Conv2d(12,16,3,padding=0,bias=False),        #conv4  8 input , 14 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:10
                                 nn.BatchNorm2d(16),                            #o/p:10
                                 nn.Dropout2d(0.04))


        self.conv5=nn.Sequential(nn.Conv2d(16,20,3,padding=0,bias=False),       #conv5  14 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:14
                                 nn.BatchNorm2d(20),                            #o/p:8
                                 nn.Dropout2d(0.04))



        #self.maxpool=nn.MaxPool2d(2,2)

        self.conv6=nn.Sequential(nn.Conv2d(20,10,1,padding=0,bias=False))       #conv6  16 input , 10 0utput and 3x3 kernel
                                                                                #RF:14
                                                                                #o/p:8


        self.conv7=nn.Sequential(nn.Conv2d(10,16,3,padding=0,bias=False),
                                 nn.ReLU(),                                     #conv6  10 input , 16 0utput and 3x3 kernel
                                 nn.BatchNorm2d(16),                            #RF:18
                                 nn.Dropout2d(0.04))                            #o/p:6


        self.GAP=nn.AvgPool2d(6)                                                # Gobal Average Pooling
                                                                                #RF:28
                                                                                #o/p:1



        self.conv8 = nn.Sequential(nn.Conv2d(16,10,1, padding=0, bias=False))   #RF:28
                                                                                #O/P:1




    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        #x=self.maxpool(x)
        x=self.conv3(x)
        x=self.maxpool(x)
        x=self.conv4(x)
        x=self.conv5(x)
        #x=self.maxpool(x)
        x=self.conv6(x)
        x=self.conv7(x)
        x=self.GAP(x)
        x=self.conv8(x)
        x=x.view(-1,10)
        return F.log_softmax(x)


#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 20, 24, 24]           2,880
              ReLU-6           [-1, 20, 24, 24]               0
       BatchNorm2d-7           [-1, 20, 24, 24]              40
         Dropout2d-8           [-1, 20, 24, 24]               0
            Conv2d-9           [-1, 12, 24, 24]             240
        MaxPool2d-10           [-1, 12, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,728
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
        Dropout2d-14           [-1, 16,

<ipython-input-10-92be57754e27>:84: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("epoches:",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoches: 1


  0%|          | 0/1875 [00:00<?, ?it/s]<ipython-input-10-92be57754e27>:84: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Loss=0.016438035294413567 Batch_id=1874 Accuracy=92.16: 100%|██████████| 1875/1875 [00:35<00:00, 53.32it/s]



Test set: Average loss: 0.0565, Accuracy: 9833/10000 (98.33%)

epoches: 2


Loss=0.03335258737206459 Batch_id=1874 Accuracy=97.23: 100%|██████████| 1875/1875 [00:33<00:00, 55.18it/s]



Test set: Average loss: 0.0360, Accuracy: 9888/10000 (98.88%)

epoches: 3


...


Test set: Average loss: 0.0184, Accuracy: 9936/10000 (99.36%)

epoches: 18


Loss=0.08415862172842026 Batch_id=1874 Accuracy=98.76: 100%|██████████| 1875/1875 [00:34<00:00, 54.12it/s]



Test set: Average loss: 0.0199, Accuracy: 9932/10000 (99.32%)

epoches: 19


Loss=0.0010608640732243657 Batch_id=1874 Accuracy=98.80: 100%|██████████| 1875/1875 [00:34<00:00, 53.75it/s]



Test set: Average loss: 0.0202, Accuracy: 9938/10000 (99.38%)

